# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
"
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk 
".


# Feladatok

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint. Melyik államban van a legtöbb, legkevesebb egyetem?


In [4]:
import pandas as pd
import matplotlib 
import numpy as np
import re

def checkingEdit(text):
  text = re.sub(']','',text)
  text = text.split('[')
  if len(text) == 1:
    return (False,text[0])
  else:
    return (True,text[0])
def checkingNumber(text):
  return any(char.isdigit() for char in text)

def checkingDuplicate(university, presentStateName, name):
  temp = university[presentStateName]
  for i in temp:
    if name == i:
      print("Duplikált elem:", name)
      return True
  return False


file = open('/content/egyetemek.txt')
university = {}
states = []
cities = []
newData = []
for row in file:
  row = re.sub('\n','',row)
  row = row.split('(')
  newData.append(row[0])

for data in newData:
  stateBool, name = checkingEdit(data)
  numberBool = checkingNumber(name)
  if stateBool and not numberBool:
    presentStateName = name;
    university.update({presentStateName:[]})
  if not stateBool and not numberBool:
    duplicatBool = checkingDuplicate(university, presentStateName, name)
    if not duplicatBool:
      temp = university[presentStateName]
      temp.append(name)
      states.append(presentStateName)
      cities.append(name)
universities = pd.DataFrame(
    {
      "state": states,
      "city": cities,
    }
)

d = universities.copy()
d['szam'] = 1
data = d.groupby('state').sum()
print(data.max())
print(data.min())

Duplikált elem: Davis 
szam    43
dtype: int64
szam    1
dtype: int64


<ipython-input-4-3bc3dfb68b41>:57: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = d.groupby('state').sum()


2. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [5]:
df_codes = pd.read_csv('/content/shortNames.csv',names = ['state', 'Code'])
universities.merge(df_codes, on = 'state', how='left')

,state,city,Code
0,﻿Alabama,Auburn,NaN
1,﻿Alabama,Florence,NaN
2,﻿Alabama,Jacksonville,NaN
3,﻿Alabama,Livingston,NaN
4,﻿Alabama,Montevallo,NaN
...,...,...,...
512,Wisconsin,River Falls,WI
513,Wisconsin,Stevens Point,WI
514,Wisconsin,Waukesha,WI
515,Wisconsin,Whitewater,WI


3. Az adatbázist integráljuk a [List of states and territories of the United States by population](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population) linken szereplő népszámlálási adatokkal és számoljuk ki államonként hány főre jut egy egyetemi város.

In [6]:
df_pop = pd.read_csv('/content/popular.csv',names = ['State', 'Popular'])

d = df[['State','Code']].copy()
d['szam'] = 1
by_state = d.groupby(['State'])
by_state = by_state.sum()
new_pop = by_state.merge(df_pop, on = 'State', how='left')
new_pop = new_pop.merge(df_codes, on = 'State', how='left')
new_pop['fo/egyet'] = new_pop['Popular']/new_pop['szam']
new_pop.head()

NameError: ignored

4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatok integrálásával, számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

In [ ]:
df_area = pd.read_csv('/content/area.csv',names = ['State', 'Area'])

d = df[['State']].copy()
d['szam'] = 1
by_state = d.groupby(['State'])
by_state = by_state.sum()
area = by_state.merge(df_area, on = 'State', how='left')
area = area.merge(df_codes, on = 'State', how='left')
area['ter/egyet'] = area['Area']//area['szam']
area.head()

5. A [példa](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/), a kivitelt, exportmennyiséget ábrázolja az Amerikai Egyesült Államok térképen, államonként lebontva ezt. Készítsünk hasonló ábrákat az egyetemek abszolút, lakoság és terület szerinti eloszlásáról is. Az ábrákat exportáljuk kép formájában.
![](https://i.ibb.co/s1zdnLY/mapplot.png)

Ábra 1. [Térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével. [Forrás](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/).

6. Számoljuk ki, hány egyetem van városonként, majd összesítve államonként. Ehhez az `egyetemek.txt` fájlban miután megkapunk egy várost, az utána következő kerek zárójelek között megszámoljuk, hány vesszővel elválasztott karakterlánc található. 
Pl. a következő sorban:	`Claremont (Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University)[5]` -> 6 vessző van a kerek zárójelek között, tehát 7 egyetem van a városban.    
Melyik államban és melyik városban van a legtöbb egyetem?